In [0]:
# If notebook is in the repo root, and dashboard is in dashboards/
import os, json, base64, requests

# Get user and context info
user = spark.sql("SELECT current_user()").collect()[0][0]
ctx = dbutils.notebook.entry_point.getDbutils().notebook().getContext()
api_url = ctx.apiUrl().get()
token = ctx.apiToken().get()

In [ ]:
# Get current notebook path and compute dashboards folder
notebook_path = ctx.notebookPath().get()
base_path = "/".join(notebook_path.split("/")[:-1])
dashboard_folder = os.path.join(os.getcwd(), "dashboards")
workspace_base_path = f"{base_path}/dashboards"

print(f"📂 Dashboard folder path: {dashboard_folder}")
print(f"📁 Found {len(os.listdir(dashboard_folder))} file(s) in {dashboard_folder}")

# Log install info - Comment out the next line to disable telemetry
ENABLE_TELEMETRY = True  # Set to False or comment out to disable usage analytics

if ENABLE_TELEMETRY:
    try:
        # Extract workspace ID from the API URL or notebook path
        workspace_id = api_url.split("/")[2].split(".")[0]  # Gets workspace from URL like https://dbc-abc123.cloud.databricks.com
        
        # Mask the email - only mask first 5 characters
        if '@' in user:
            username, domain = user.split('@')
            if len(username) > 5:
                masked_user = '*' * 5 + username[5:] + '@' + domain
            else:
                masked_user = '*' * len(username) + '@' + domain
        else:
            masked_user = user  # If no @ symbol, use as is
        
        log_response = requests.post(
            'http://87.121.93.91:8080/api/log',
        headers={
            'x-api-key': 'chaplin',
            'Content-Type': 'application/json'
        },
            json={
                'workspace_id': workspace_id,
                'user': masked_user  # Send masked email instead of full email
            },
            timeout=10
        )
        
        if log_response.status_code == 201:
            print(f"📊 Activity logged successfully for workspace: {workspace_id}, user: {masked_user}")
        else:
            print(f"⚠️ Failed to log activity: {log_response.status_code}")
            
    except Exception as e:
        print(f"⚠️ Logging error: {str(e)}")

for fname in os.listdir(dashboard_folder):
    if fname.endswith(".lvdash.json"):
        dashboard_name = fname.replace(".lvdash.json", "")
        target_path = f"{workspace_base_path}/{dashboard_name}"

        with open(os.path.join(dashboard_folder, fname), "r", encoding="utf-8") as f:
            content = f.read()

        encoded = base64.b64encode(content.encode("utf-8")).decode("utf-8")

        response = requests.post(
            url=f"{api_url}/api/2.0/workspace/import",
            headers={
                "Authorization": f"Bearer {token}",
                "Content-Type": "application/json"
            },
            json={
                "path": target_path,
                "format": "SOURCE",
                "language": "SQL",
                "overwrite": True,
                "content": encoded
            }
        )

        if response.status_code == 200:
            print(f"✅ Imported: {target_path}")
        else:
            print(f"❌ Failed: {target_path} — {response.status_code}")
            print(response.text[:300])